In [83]:
import csv
import pandas as pd

# Lire le fichier et réécrire une version nettoyée
input_file = "G-D.csv"
output_file = "G-D_cleaned.csv"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8", newline="") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # Entoure tous les champs de guillemets
    for row in reader:
        writer.writerow(row)

# Charger le fichier nettoyé
df = pd.read_csv(output_file, sep=",", encoding="utf-8")
print("Aperçu des données :")
df.head()

Aperçu des données :


/tmp/ipykernel_10259/2078072913.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_file, sep=",", encoding="utf-8")


,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23 03:50:13,4680.0,2643.0,6232.0,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30 17:41:41,770.0,1145.0,9166.0,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False
2,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,576.0,1.0,0.0,False,2023-03-01 23:59:42,#Ethereum price update: \n\n#ETH $1664.02 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe\n\n#bitcoin #blockchai...,2010-03-26 10:15:26,236.0,1829.0,2195.0,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here\nAvailable on ios a...,['Bitcoin'],Twitter for Android,False
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16 09:57:56,12731.0,46.0,134.0,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...,"['Bitcoin', 'BTC']",Twitter Web App,False


In [84]:
# Vérifier les valeurs manquantes
print("Nombre de valeurs manquantes par colonne :")
print(df.isna().sum())

Nombre de valeurs manquantes par colonne :
user_name              61
user_location       89369
user_description    14834
user_created         3770
user_followers       4577
user_friends         4577
user_favourites      4577
user_verified        4577
date                 4577
text                 4577
hashtags             5384
source               5384
is_retweet           5443
dtype: int64


In [85]:
# Afficher les types de données
print("Types de données des colonnes :")
print(df.dtypes)

Types de données des colonnes :
user_name            object
user_location        object
user_description     object
user_created         object
user_followers      float64
user_friends        float64
user_favourites      object
user_verified        object
date                 object
text                 object
hashtags             object
source               object
is_retweet           object
dtype: object


convert field to numeric and bool

In [86]:

df["user_favourites"] = pd.to_numeric(df["user_favourites"], errors="coerce")

# Convertir les colonnes de date
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["user_created"] = pd.to_datetime(df["user_created"], errors="coerce")


# Ensure the 'user_verified' column contains only boolean-like values
df['user_verified'] = df['user_verified'].apply(lambda x: True if x in [True, 'True', 1] else False if x in [False, 'False', 0] else None)

df['user_verified'] = df['user_verified'].astype('boolean')
df['is_retweet'] = df['is_retweet'].astype('boolean') 

verify if the change is happaned

In [87]:
print("Types de données après conversion :")
print(df.dtypes)

Types de données après conversion :
user_name                   object
user_location               object
user_description            object
user_created        datetime64[ns]
user_followers             float64
user_friends               float64
user_favourites            float64
user_verified              boolean
date                datetime64[ns]
text                        object
hashtags                    object
source                      object
is_retweet                 boolean
dtype: object


drop cloum without text and date
remplace the othor values

In [88]:
df = df.dropna(subset=["text", "date"])

# Remplacer les valeurs manquantes dans hashtags par "[]"
df["hashtags"] = df["hashtags"].fillna("[]")

# Remplacer les valeurs manquantes dans les colonnes liées à l'utilisateur
df["user_name"] = df["user_name"].fillna("Unknown")
df["user_location"] = df["user_location"].fillna("")
df["user_description"] = df["user_description"].fillna("")
df["user_followers"] = df["user_followers"].fillna(0)
df["user_friends"] = df["user_friends"].fillna(0)
df["user_favourites"] = df["user_favourites"].fillna(0)
df["user_verified"] = df["user_verified"].map({"True": True, "False": False}).fillna(False)
df["is_retweet"] = df["is_retweet"].map({"True": True, "False": False}).fillna(False)
df["source"] = df["source"].fillna("Unknown")

print(df.isna().sum())

user_name           0
user_location       0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
dtype: int64


/tmp/ipykernel_10259/3664603681.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["user_verified"] = df["user_verified"].map({"True": True, "False": False}).fillna(False)
/tmp/ipykernel_10259/3664603681.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["is_retweet"] = df["is_retweet"].map({"True": True, "False": False}).fillna(False)


In [57]:
print("Date minimale :", df["date"].min())
print("Date maximale :", df["date"].max())

Date minimale : 2023-02-25 20:49:09
Date maximale : 2023-03-05 23:59:56


In [66]:
import pandas as pd

# S'assurer que la colonne date est au format datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")

# Vérifier la plage de dates
print("Plage de dates des tweets :")
print("Date minimale :", df["date"].min())
print("Date maximale :", df["date"].max())
print("Nombre total de tweets :", len(df))

# Vérifier la distribution des tweets par heure (optionnel, pour voir la répartition)
df["hour"] = df["date"].dt.hour
print("\nNombre de tweets par heure :")
print(df["hour"].value_counts().sort_index())

Plage de dates des tweets :
Date minimale : 2023-02-25 20:49:09
Date maximale : 2023-03-05 23:59:56
Nombre total de tweets : 169761

Nombre de tweets par heure :
hour
0     4856
1     5208
2     4490
3     4453
4     4311
5     4692
6     4735
7     4959
8     6358
9     5728
10    6446
11    7070
12    7840
13    8265
14    8585
15    9457
16    9811
17    9492
18    9878
19    9331
20    9046
21    9002
22    8185
23    7563
Name: count, dtype: int64


gener data from banance with the same date as our data

In [90]:
import pandas as pd
from binance.client import Client
from datetime import datetime

# Remplace par tes clés API
api_key = "TLXuzjmtV05vOkRYBaeQXWg3p3A6FbsEEVur2ixrJB7bBjY76G0IS1sp4ALhKQjq"  # Remplace par ta clé API
api_secret = "7DNgiq61ajYq7Ul6l8wW5sfUqgSr742PHHqMZgh3Gcn0hnrssvMqKWF80XhKsUtT"  # Remplace par ta clé secrète

# Initialiser le client Binance
client = Client(api_key, api_secret)

# Définir la paire de trading et la période
symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
start_time = datetime(2023, 2, 25, 0, 0, 0)  # 5 février 2023 00:00:00
end_time = datetime(2023, 3, 5, 23, 59, 59)  # 5 mars 2023 23:59:59
start_time_ms = int(start_time.timestamp() * 1000)
end_time_ms = int(end_time.timestamp() * 1000)

# Récupérer les données historiques
klines = client.get_historical_klines(
    symbol=symbol,
    interval=interval,
    start_str=start_time_ms,
    end_str=end_time_ms
)

# Convertir les données en DataFrame
df_merge_price = pd.DataFrame(klines, columns=[
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
])

# Convertir les timestamps en format datetime
df_merge_price["date"] = pd.to_datetime(df_merge_price["open_time"], unit="ms")

# Sélectionner les colonnes pertinentes
df_merge_price = df_merge_price[["date", "close"]]
df_merge_price.rename(columns={"close": "price"}, inplace=True)
df_merge_price["price"] = df_merge_price["price"].astype(float)

# Vérifier les données
print("Aperçu des données de prix collectées :")
print(df_merge_price.head())

Aperçu des données de prix collectées :
                 date     price
0 2023-02-25 00:00:00  23179.84
1 2023-02-25 00:01:00  23181.07
2 2023-02-25 00:02:00  23172.72
3 2023-02-25 00:03:00  23182.30
4 2023-02-25 00:04:00  23186.56


Merge data

In [91]:
# Arrondir les dates des tweets à la minute la plus proche
df["date_bucket"] = df["date"].dt.floor("T")

# Inspect the columns before merging
print("Columns in df before merge:", df.columns)
print("Columns in df_merge_price:", df_merge_price.columns)

# Perform the merge
df = df.merge(df_merge_price, left_on="date_bucket", right_on="date", how="left")

# Drop unnecessary columns and rename appropriately
df = df.drop(columns=["date_y"])  # Drop the duplicate 'date' column from df_merge_price
df.rename(columns={"date_x": "date", "price": "Bitcoin_Price"}, inplace=True)

# Remove duplicate columns if they exist
df = df.loc[:, ~df.columns.duplicated()]

# Check the columns after cleanup
print("Columns after merge and cleanup:", df.columns)

# Verify rows with missing Bitcoin_Price
print(df.isna().sum())
print(len(df))

/tmp/ipykernel_10259/2147767952.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df["date_bucket"] = df["date"].dt.floor("T")


Columns in df before merge: Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'is_retweet', 'date_bucket'],
      dtype='object')
Columns in df_merge_price: Index(['date', 'price'], dtype='object')
Columns after merge and cleanup: Index(['user_name', 'user_location', 'user_description', 'user_created',
       'user_followers', 'user_friends', 'user_favourites', 'user_verified',
       'date', 'text', 'hashtags', 'source', 'is_retweet', 'date_bucket',
       'Bitcoin_Price'],
      dtype='object')
user_name           0
user_location       0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
date_bucket         0
Bitcoin_Price       0
dtype: int64
169761


Merger  data of BINANCE with G-D.csv and save finale data into(G-D_final.csv)

In [93]:
# Enregistrer le DataFrame final dans un fichier CSV
output_file = "G-D_final.csv"
df.to_csv(output_file, index=False, encoding="utf-8")
print(f"Fichier final enregistré sous : {output_file}")
# Afficher les premières lignes du DataFrame final

Fichier final enregistré sous : G-D_final.csv


In [95]:
df.to_csv("G-D_final.csv", index=False, encoding="utf-8")
# Afficher les premières lignes du DataFrame final          
print("Aperçu des données finales :")
df.head()
print(len(df))

Aperçu des données finales :
169761
